In [4]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from dotenv import load_dotenv
import os
from ast import literal_eval
import json
from tqdm import tqdm
import pickle
import random

from yt_rag.agent import llm, build_prompt
from yt_rag.build_index import build_index, create_embeddings
from yt_info.yt_video_data import Video, get_video_transcript


load_dotenv()

OLLAMA_URL = os.getenv("LOCAL_OLLAMA_URL")
CHANNELS = literal_eval(os.getenv("YT_CHANNELS"))
ES_URL = os.getenv("LOCAL_ES_URL")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")

ollama_client = OpenAI(base_url=OLLAMA_URL, api_key="ollama")
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

es_client = Elasticsearch(ES_URL)

/home/pedro-loka/.cache/pypoetry/virtualenvs/yt-summaries-019raoQi-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
with open('../data/yt_videos_details.pkl', 'rb') as f:
    videos = pickle.load(f)


with open('../data/rag_retrieval.pkl', 'rb') as f:
    results = pickle.load(f)

In [6]:
prompt_llm = """
You are a user of a cooking and recipe assistant app.
Generate 5 general questions related to the cooking techniques, ingredients, or recipes discussed in the provided video.
The questions should be clear, concise, and relevant to the video's content.
Avoid being overly specific about particular ingredients or using too many details from the title and description.
Do not mention or reference the video title, description, or any URLs.
Focus on broader cooking themes that would be of interest to a home cook.

The video information:

title: {title}
description: {description}

Output the result as a JSON object without using code blocks:

{{"questions": ["question1", "question2", "question3", "question4", "question5"]}}
""".strip()

## Generate questions

In [7]:
def generate_questions(video):
    prompt = prompt_llm.format(**video.__dict__)

    response = llm(prompt, client=ollama_client)

    return response

In [8]:
results = {}
failed_video_ids = {}

In [51]:
for video in tqdm(videos): 
    if video.video_id in results:
        continue

    try:
        questions_raw = generate_questions(video)
    except Exception as err:
        print(f"Failed {video.video_id}", err)
        continue


    try:
        questions = json.loads(questions_raw.replace("json", "").replace("`", "").strip())
    except json.JSONDecodeError:
        # print(f"JSON fail for {video.video_id}")
        failed_video_ids[video.video_id] = questions_raw
        continue        
    except Exception as err:
        # print(f"Failed {video.video_id}", err)
        failed_video_ids[video.video_id] = questions_raw
        continue
    else:
        
        try:
            results[video.video_id] = questions['questions']
        except KeyError:
            failed_video_ids[video.video_id] = questions_raw
            continue
        

100%|██████████| 475/475 [00:57<00:00,  8.21it/s]


## Retrieval Evalutaion

In [65]:
def elastic_search_knn(query):
    field = "title_description_vector"
    
    vector = embedding_model.encode(query)
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 20,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["title", "is_short", "description", "course", "video_id"],
    }

    es_results = es_client.search(index=ES_INDEX_NAME, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [66]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['video_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [67]:
ground_truth = [{"id": k, "question": _v} for k, v in results.items() for _v in v]

### Title + description and Phi3-Mini

In [ ]:
embeddings = create_embeddings(videos)
build_index(videos, embeddings, ES_INDEX_NAME, es_client=es_client)

In [68]:
evaluate(ground_truth, lambda q: elastic_search_knn(q['question']))

100%|██████████| 2371/2371 [01:32<00:00, 25.58it/s]


{'hit_rate': 0.742724588781105, 'mrr': 0.5268124092038056}

## RAG Evaluation

In [ ]:
prompt_rag_eval = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [ ]:
def get_answer(query):
    es_videos = elastic_search_knn(query)
    es_videos = [Video(**video) for video in es_videos]
    transcripts = []
    for video in es_videos[:1]:
        transcript = get_video_transcript(video)
        transcripts.append("\n".join([line["text"] for line in transcript]))

    prompt = build_prompt(query, es_videos[:1], transcripts)
    answer = llm(prompt, client=ollama_client)

    return answer



In [ ]:
sample = random.sample(ground_truth, 200)
rag_evals = []
for entry in tqdm(sample):
    question = entry["question"]
    
    llm_answer = get_answer(question)
    rag_eval_prompt = prompt_rag_eval.format(question=question, answer_llm=llm_answer)   

    rag_llm_eval = llm(rag_eval_prompt, client=ollama_client)
    
    rag_evals.append((entry, rag_llm_eval))
    break
    

In [ ]:
[i for i in videos if i.video_id == "vaX2Dv6TaSk"]